## V. How the number of hookups is related to student's characteristics
Many colleges and universities in U.S. have "Greek letter organizations", i.e., fraternities and sororities. These organizations are often known for their social events, especially parties with alcohol. We have unique data from one U.S. university in which fraternity and sorority members were surveyed. Please use the data "collegehookup" downloaded from CEIBA to study how the number of hookups is related to student's characteristics, i.e., provide estimation result and interpretation. You should use the Poission regression as the dependent variable is count number. Try to code the maximum likelihood
estimator by yourself.

Following the previous question, please use the same data to study if there exist peer effects from the same fraternity (sorority) house on hookup.

In [ ]:
import numpy as np
from numpy import exp
%matplotlib inline
from scipy.special import factorial
import pandas as pd
import statsmodels.api as sm
from statsmodels.api import Poisson
from scipy import stats
from scipy.stats import norm
from statsmodels.iolib.summary2 import summary_col


import io
from google.colab import files
uploaded = files.upload()

#change the filename to match the uploaded file's
df = pd.read_csv(io.BytesIO(uploaded['collegehookup.csv']))
df.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Saving collegehookup.csv to collegehookup.csv


,greek_group,greek_house,hookup_sum,Gender,Age,Hisp,Black,Asian,Native,Mideast,BMI,BMI2,college_dad,college_mom,hookup_highschool,ParentsDivorce,Siblings
0,1,2,4,1,19,0,0,0,0,0,20.17332,406.9630,1,1,2,2,2
1,1,2,0,1,18,0,0,1,0,0,22.95510,526.9368,1,1,0,2,1
2,1,2,4,1,18,0,0,0,0,0,23.01031,529.4746,1,1,15,2,2
3,1,2,1,1,19,0,0,1,0,0,20.98507,440.3734,0,0,1,2,1
4,1,2,0,1,19,0,0,0,0,0,21.69753,470.7828,1,1,5,1,1


In [ ]:
class PoissonRegression:

    def __init__(self, y, X, β):
        self.X = X
        self.n, self.k = X.shape
        # Reshape y as a n_by_1 column vector
        self.y = y.reshape(self.n,1)
        # Reshape β as a k_by_1 column vector
        self.β = β.reshape(self.k,1)

    def μ(self):
        return np.exp(self.X @ self.β)

    def logL(self):
        y = self.y
        μ = self.μ()
        return np.sum(y * np.log(μ) - μ - np.log(factorial(y)))

    def G(self):
        y = self.y
        μ = self.μ()
        return X.T @ (y - μ)

    def H(self):
        X = self.X
        μ = self.μ()
        return -(X.T @ (μ * X))

In [ ]:
def newton_raphson(model, tol=1e-10, max_iter=1000, display=True):

    i = 0
    error = 100  # Initial error value

    # Print header of output
    if display:
        header = f'{"Iteration_k":<13}{"Log-likelihood":<16}{"θ":<60}'
        print(header)
        print("-" * len(header))

    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(np.abs(error) > tol) and i < max_iter:
        H, G = model.H(), model.G()
        β_new = model.β - (np.linalg.inv(H) @ G)
        error = β_new - model.β
        model.β = β_new

        # Print iterations
        if display:
            β_list = [f'{t:.3}' for t in list(model.β.flatten())]
            update = f'{i:<13}{model.logL():<16.8}{β_list}'
            print(update)

        i += 1

    print(f'Number of iterations: {i}')
    print(f'β_hat = {model.β.flatten()}')

    # Return a flat array for β (instead of a k_by_1 column vector)
    return model.β.flatten()

In [ ]:
y = df['hookup_sum'].values
X = df[['Gender','Age','greek_house']].values

# Take a guess at initial βs
init_β = np.array([0.1, 0.1, 0.1])

# Create an object with Poisson model values
poi = PoissonRegression(y, X, β=init_β)

# Use newton_raphson to find the MLE
β_hat = newton_raphson(poi, display=True)

Iteration_k  Log-likelihood  θ                                                           
-----------------------------------------------------------------------------------------
0            -1673.3097      ['0.114', '0.0709', '0.0818']
1            -1151.1753      ['0.157', '0.0626', '0.0454']
2            -1056.7007      ['0.217', '0.0713', '0.00188']
3            -1048.4907      ['0.219', '0.077', '-0.0172']
4            -1048.3897      ['0.214', '0.0776', '-0.0193']
5            -1048.3897      ['0.214', '0.0776', '-0.0193']
6            -1048.3897      ['0.214', '0.0776', '-0.0193']
7            -1048.3897      ['0.214', '0.0776', '-0.0193']
Number of iterations: 8
β_hat = [ 0.21397624  0.07760284 -0.01933411]


In [ ]:
stats_poisson = Poisson(y, X).fit()
print(stats_poisson.summary())

Optimization terminated successfully.
         Current function value: 4.499527
         Iterations 8
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:                  233
Model:                        Poisson   Df Residuals:                      230
Method:                           MLE   Df Model:                            2
Date:                Thu, 07 May 2020   Pseudo R-squ.:               -0.002912
Time:                        09:56:14   Log-Likelihood:                -1048.4
converged:                       True   LL-Null:                       -1045.3
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2140      0.102      2.094      0.036       0.014       0.414
x2             0.0776      0.

As above, we can see that there is a positive relationship between gender and hooking up, and\
also age and hooking up. However, it seems that there is a negative relationship between\
house and hooking up.